In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import Prepare_Data
import pyarrow 
import numpy as np

#autoreload download #pip install ipython
%load_ext autoreload
%autoreload 2

file_path_id_test="./data_test/top_5_percent_ids.csv"
# Sti til usa_dsf.parquet
file_path_usa_dsf_test = "./data_test/usa_dsf_test.parquet"
file_path_usa_test = "./data_test/usa_test.parquet"
file_path_cluster_labels = "./Data/Cluster Labels.csv"
file_path_factor_details = "./Data/Factor Details.xlsx"


file_path_usa = "./Data/usa.parquet"
file_path_rvol_252 = "./Data/rvol_252.csv"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
features = [
  "age",                 "aliq_at",             "aliq_mat",            "ami_126d",
  "at_be",               "at_gr1",              "at_me",               "at_turnover",
  "be_gr1a",             "be_me",               "beta_60m",            "beta_dimson_21d",
  "betabab_1260d",       "betadown_252d",       "bev_mev",             "bidaskhl_21d",
  "capex_abn",           "capx_gr1",            "capx_gr2",            "capx_gr3",
  "cash_at",             "chcsho_12m",          "coa_gr1a",            "col_gr1a",
  "cop_at",              "cop_atl1",            "corr_1260d",          "coskew_21d",
  "cowc_gr1a",           "dbnetis_at",          "debt_gr3",            "debt_me",
  "dgp_dsale",           "div12m_me",           "dolvol_126d",         "dolvol_var_126d",
  "dsale_dinv",          "dsale_drec",          "dsale_dsga",          "earnings_variability",
  "ebit_bev",            "ebit_sale",           "ebitda_mev",          "emp_gr1",
  "eq_dur",              "eqnetis_at",          "eqnpo_12m",           "eqnpo_me",
  "eqpo_me",             "f_score",             "fcf_me",              "fnl_gr1a",
  "gp_at",               "gp_atl1",             "ival_me",             "inv_gr1",
  "inv_gr1a",            "iskew_capm_21d",      "iskew_ff3_21d",       "iskew_hxz4_21d",
  "ivol_capm_21d",       "ivol_capm_252d",      "ivol_ff3_21d",        "ivol_hxz4_21d",
  "kz_index",            "lnoa_gr1a",           "lti_gr1a",            "market_equity",
  "mispricing_mgmt",     "mispricing_perf",     "ncoa_gr1a",           "ncol_gr1a",
  "netdebt_me",          "netis_at",            "nfna_gr1a",           "ni_ar1",
  "ni_be",               "ni_inc8q",            "ni_ivol",             "ni_me",
  "niq_at",              "niq_at_chg1",         "niq_be",              "niq_be_chg1",
  "niq_su",              "nncoa_gr1a",          "noa_at",              "noa_gr1a",
  "o_score",             "oaccruals_at",        "oaccruals_ni",        "ocf_at",
  "ocf_at_chg1",         "ocf_me",              "ocfq_saleq_std",      "op_at",
  "op_atl1",             "ope_be",              "ope_bel1",            "opex_at",
  "pi_nix",              "ppeinv_gr1a",         "prc",                 "prc_highprc_252d",
  "qmj",                 "qmj_growth",          "qmj_prof",            "qmj_safety",
  "rd_me",               "rd_sale",             "rd5_at",              "resff3_12_1",
  "resff3_6_1",          "ret_1_0",             "ret_12_1",            "ret_12_7",
  "ret_3_1",             "ret_6_1",             "ret_60_12",           "ret_9_1",
  "rmax1_21d",           "rmax5_21d",           "rmax5_rvol_21d",      "rskew_21d",
  "rvol_21d",            "sale_bev",            "sale_emp_gr1",        "sale_gr1",
  "sale_gr3",            "sale_me",             "saleq_gr1",           "saleq_su",
  "seas_1_1an",          "seas_1_1na",          "seas_11_15an",        "seas_11_15na",
  "seas_16_20an",        "seas_16_20na",        "seas_2_5an",          "seas_2_5na",
  "seas_6_10an",         "seas_6_10na",         "sti_gr1a",            "taccruals_at",
  "taccruals_ni",        "tangibility",         "tax_gr1a",            "turnover_126d",
  "turnover_var_126d",   "z_score",             "zero_trades_126d",    "zero_trades_21d",
  "zero_trades_252d",
  "rvol_252d"
]

In [5]:
import pandas as pd

def load_and_prepare_data(file_path, features):
    """
    Indlæser og forbereder rådata fra en Parquet-fil.
    
    Args:
        file_path (str): Stien til Parquet-filen (USA testfil).
        features (list): Liste over features, der skal indlæses.
    
    Returns:
        pd.DataFrame: DataFrame med forberedte data.
    """
    # Liste over kolonner, vi gerne vil have
    desired_cols = ["id", "eom", "sic", "ff49", "size_grp", "me", "crsp_exchcd", "rvol_252d", "dolvol_126d"] + features
    
    # Tjek hvilke kolonner der faktisk findes i Parquet-filen
    actual_cols = pd.read_parquet(file_path, engine="pyarrow").columns
    cols_to_load = [col for col in desired_cols if col in actual_cols]
    
    if not cols_to_load:
        raise ValueError("Ingen af de ønskede kolonner findes i Parquet-filen.")
    
    # Indlæs kun de kolonner, der faktisk findes
    data = pd.read_parquet(file_path, engine="pyarrow", columns=cols_to_load)
    
    # Filtrér observationer
    data = data[data["id"] <= 99999]
    
    # Konverter datoformat for 'eom'
    data["eom"] = pd.to_datetime(data["eom"], errors="coerce")
    
    return data


In [9]:
data = load_and_prepare_data(file_path_usa_test, features)
data.head()

,id,eom,sic,size_grp,me,rvol_252d,dolvol_126d,age,aliq_at,aliq_mat,...,taccruals_ni,tangibility,tax_gr1a,turnover_126d,turnover_var_126d,z_score,zero_trades_126d,zero_trades_21d,zero_trades_252d,rvol_252d
0,10104,2010-01-31,7372,mega,115558.73052,0.019544,6.798938e+08,301,0.526422,0.232151,...,-0.378191,0.384579,-0.000151,0.006062,0.439375,4.269989,0.002792,0.002780,0.002652,0.019544
1,10104,2010-02-28,7372,mega,123619.74809,0.018949,7.094750e+08,302,0.526422,0.249602,...,-0.378191,0.384579,-0.000151,0.006243,0.425637,4.269989,0.002704,0.002416,0.002676,0.018949
2,10104,2010-03-31,7372,mega,129040.82501,0.015241,6.982804e+08,303,0.570962,0.230516,...,-0.086350,0.384579,0.000186,0.005967,0.356592,4.405333,0.002753,0.002751,0.002807,0.015241
3,10104,2010-04-30,7372,mega,130101.18774,0.014489,6.802223e+08,304,0.570962,0.218079,...,-0.086350,0.384579,0.000186,0.005590,0.344846,4.405333,0.002935,0.003165,0.002890,0.014489
4,10104,2010-05-31,7372,mega,113436.81847,0.015136,7.368972e+08,305,0.570962,0.215020,...,-0.086350,0.384579,0.000186,0.006023,0.361901,4.405333,0.002833,0.002529,0.002844,0.015136


In [14]:
def calculate_derived_columns(data, pi):
    """
    Beregner afledte kolonner.
    
    Args:
        data (pd.DataFrame): Input DataFrame.
        pi (float): En skalar værdi, der bruges i beregningen af lambda.
    
    Returns:
        pd.DataFrame: DataFrame med nye kolonner.
    """
    # Kontrollér inputtyper
    if not isinstance(pi, (int, float)):
        raise ValueError("'pi' skal være en numerisk værdi (int eller float).")

    data = data.loc[:, ~data.columns.duplicated()]
    
    # Kontrollér for nødvendige kolonner
    required_cols = ["dolvol_126d", "rvol_252d"]
    for col in required_cols:
        if col not in data.columns:
            raise KeyError(f"Kolonnen '{col}' mangler i DataFrame.")

    # Debugging: Tjek for null-værdier
    for col in required_cols:
        if data[col].isnull().any():
            raise ValueError(f"Kolonnen '{col}' indeholder manglende værdier.")
    
    # Debugging: Tjek for 0-værdier i 'dolvol_126d'
    if (data["dolvol_126d"] == 0).any():
        raise ValueError("Kolonnen 'dolvol_126d' indeholder værdier lig 0, hvilket vil føre til divisionsfejl.")

    # Beregn afledte kolonner
    print("Beregner 'dolvol', 'lambda' og 'rvol_m'...")
    data["dolvol"] = data["dolvol_126d"]  # Kopiér dolvol_126d til dolvol
    data["lambda"] = 2 / data["dolvol"] * pi  # Beregn lambda
    data["rvol_m"] = data["rvol_252d"] * np.sqrt(21)  # Beregn rvol_m
    
    print("Beregnede kolonner 'dolvol', 'lambda' og 'rvol_m' succesfuldt.")
    return data

In [16]:
data=calculate_derived_columns(data, 0.1)

Beregner 'dolvol', 'lambda' og 'rvol_m'...
Beregnede kolonner 'dolvol', 'lambda' og 'rvol_m' succesfuldt.


In [ ]:
def merge_data(data, data_ret_ld1, wealth):
    """
    Slår data sammen med afkast og wealth.
    
    Args:
        data (pd.DataFrame): Hoveddataset.
        data_ret_ld1 (pd.DataFrame): Afkastdata.
        wealth (pd.DataFrame): Wealth-data.
    
    Returns:
        pd.DataFrame: Kombineret DataFrame.
    """
    # Merge afkastdata
    data = pd.merge(data, data_ret_ld1, on=["id", "eom"], how="left")
    
    # Merge wealth data
    wealth["eom"] = wealth["eom"] + pd.offsets.MonthBegin(2) - pd.Timedelta(days=1)
    data = pd.merge(data, wealth[["eom", "mu_ld1"]].rename(columns={"mu_ld1": "mu_ld0"}), on="eom", how="left")
    
    return data


In [ ]:
def screen_data(data, settings):
    """
    Filtrerer data baseret på forskellige kriterier.
    
    Args:
        data (pd.DataFrame): Input DataFrame.
        settings (dict): Indstillinger for screens.
    
    Returns:
        pd.DataFrame: Filtreret DataFrame.
    """
    # Start antal observationer og markedsværdi
    n_start = len(data)
    me_start = data["me"].sum()
    
    # NYSE screen
    if settings["screens"]["nyse_stocks"]:
        data = data[data["crsp_exchcd"] == 1]
    
    # Dato screen
    data = data[(data["eom"] >= settings["screens"]["start"]) & (data["eom"] <= settings["screens"]["end"])]
    
    # Fjern observationer med manglende eller ugyldige værdier
    data = data[data["me"].notna() & data["tr_ld0"].notna() & data["tr_ld1"].notna()]
    data = data[data["dolvol"].notna() & (data["dolvol"] > 0)]
    data = data[data["sic"].notna()]
    
    return data


In [ ]:
def standardize_and_impute_features(data, features, settings):
    """
    Standardiserer og imputerer features.
    
    Args:
        data (pd.DataFrame): Input DataFrame.
        features (list): Liste over features.
        settings (dict): Indstillinger for standardisering og imputering.
    
    Returns:
        pd.DataFrame: DataFrame med standardiserede og imputerede features.
    """
    # Standardisering
    if settings["feat_prank"]:
        for f in features:
            data[f] = data[f].rank(pct=True)
            data[f] = data[f].fillna(0)  # Sæt manglende værdier til 0
    
    # Imputering
    if settings["feat_impute"]:
        for f in features:
            if settings["feat_prank"]:
                data[f] = data[f].fillna(0.5)  # Median for standardiserede værdier
            else:
                data[f] = data.groupby("eom")[f].transform(lambda x: x.fillna(x.median()))
    
    return data


In [ ]:
def classify_industries(data):
    """
    Klassificerer observationer baseret på SIC-koder.
    
    Args:
        data (pd.DataFrame): Input DataFrame.
    
    Returns:
        pd.DataFrame: DataFrame med klassificerede industrier.
    """
    conditions = [
        data["sic"].between(100, 999) | data["sic"].between(2000, 2399),
        data["sic"].between(2500, 2519) | data["sic"].between(3630, 3659),
        data["sic"].between(2520, 2589) | data["sic"].between(2600, 2699),
        data["sic"].between(1200, 1399) | data["sic"].between(2900, 2999)
    ]
    choices = ["NoDur", "Durbl", "Manuf", "Enrgy"]
    data["ff12"] = np.select(conditions, choices, default="Other")
    
    return data


In [ ]:
def validate_and_finalize(data, pf_set):
    """
    Validerer og afslutter datasættet.
    
    Args:
        data (pd.DataFrame): Input DataFrame.
        pf_set (dict): Indstillinger for validering.
    
    Returns:
        pd.DataFrame: Valideret DataFrame.
    """
    lb = pf_set["lb_hor"] + 1
    data["eom_lag"] = data.groupby("id")["eom"].shift(lb)
    data["month_diff"] = ((data["eom"] - data["eom_lag"]).dt.days / 30).round()
    data["valid_data"] = (data["month_diff"] == lb) & data["month_diff"].notna()
    data = data[data["valid_data"]]
    
    return data


In [42]:
def prepare_daily_returns(file_path, data):
    """
    Forbereder daglige afkastdata.
    
    Args:
        file_path (str): Stien til CSV-filen med daglige data. usa_dsf
        chars (pd.DataFrame): Filtreret DataFrame med gyldige observationer.
    
    Returns:
        pd.DataFrame: DataFrame med daglige afkast.
    """
    usecols=["id", "date", "ret_exc"]
    daily = pd.read_parquet(file_path, engine="pyarrow", columns=usecols)
    daily = daily[daily["ret_exc"].notna() & daily["id"].isin(data["id"].unique())]
    daily["date"] = pd.to_datetime(daily["date"], format="%Y%m%d")
    daily["eom"] = daily["date"] + pd.offsets.MonthEnd(0)
    
    return daily


In [43]:
prepare_daily_returns("./data_test/usa_dsf_test.parquet", data)

,id,date,ret_exc,eom
0,10104,2010-02-01,0.006938,2010-02-28
1,10104,2010-02-02,0.023255,2010-02-28
2,10104,2010-02-03,-0.000421,2010-02-28
3,10104,2010-02-04,-0.026948,2010-02-28
4,10104,2010-02-05,0.019039,2010-02-28
...,...,...,...,...
602507,92655,2023-12-22,0.000624,2023-12-31
602508,92655,2023-12-26,-0.000741,2023-12-31
602509,92655,2023-12-27,0.005104,2023-12-31
602510,92655,2023-12-28,0.003833,2023-12-31
